# Introduction

The "Affairs.csv" is an infidelity data, known as Fair's Affairs. Cross-section data from a survey conducted in 1969.
Your task is to investigate this data.

## Features

| Variable       | Description                                                        |
| -------------- | ------------------------------------------------------------------ |
| affairs        | How often engaged in extramarital sexual intercourse during the past year? |
| gender         | Gender                                                             |
| age            | Age in years. Coding: 17.5 = under 20, 22 = 20–24, 27 = 25–29, 32 = 30–34, 37 = 35–39, 42 = 40–44, 47 = 45–49, 52 = 50–54, 57 = 55 or over. |
| yearsmarried   | Number of years married. Coding: 0.125 = 3 months or less, 0.417 = 4–6 months, 0.75 = 6 months–1 year, 1.5 = 1–2 years, 4 = 3–5 years, 7 = 6–8 years, 10 = 9–11 years, 15 = 12 or more years. |
| children       | Are there children in the marriage?                                 |
| religiousness  | Religiousness level. Coding: 1 = anti, 2 = not at all, 3 = slightly, 4 = somewhat, 5 = very. |
| education      | Level of education. Coding: 9 = grade school, 12 = high school graduate, 14 = some college, 16 = college graduate, 17 = some graduate work, 18 = master's degree, 20 = Ph.D., M.D., or other advanced degree. |
| occupation     | Occupation according to Hollingshead classification (reverse numbering) |
| rating         | Self-rating of marriage. Coding: 1 = very unhappy, 2 = somewhat unhappy, 3 = average, 4 = happier than average, 5 = very happy. |


A bit more clarification on occupation (Hollingshead classification)

| Code | Meaning                                                              |
| ---- | -------------------------------------------------------------------- |
| 1    | Unemployed and Unemployable                                        |
| 2    | Partly Unemployed and Partly Employed                               |
| 3    | Unskilled Manual Workers                                            |
| 4    | Semi-Skilled Workers                                                |
| 5    | Skilled and Technical Workers                                       |
| 6    | Managers and Supervisors                                            |
| 7    | Prestigious Professionals                                           |


# Coding Part 1

This part is a guided coding part

## Import Libraries

In [2]:
import pandas as pd
import plotly.graph_objects as go 
import plotly.express as px
import statsmodels.api as sm

## Data preprocessing

### Load the data and display the first 5 rows of the dataset using head function

In [26]:
df = pd.read_csv("Affairs.csv")
print(df.head())

   Unnamed: 0  affairs  gender   age  yearsmarried children  religiousness  \
0           4        0    male  37.0         10.00       no              3   
1           5        0  female  27.0          4.00       no              4   
2          11        0  female  32.0         15.00      yes              1   
3          16        0    male  57.0         15.00      yes              5   
4          23        0    male  22.0          0.75       no              2   

   education  occupation  rating  
0         18           7       4  
1         14           6       4  
2         12           1       4  
3         18           6       5  
4         17           6       3  


### Delete the column Unnamed:0, these are the id's of the participants.

In [27]:
df = df.drop("Unnamed: 0", axis = 1)

### Change the values of the columns:

gender: code male as 1, female as 0.

children: code yes as 1, no as 0.

For column affairs, make a new column affairs_binary
where 0 from affairs becomes 0 and all other values becomes 1.
For coding part 1 we will be only using affairs_binary.

In [28]:
df["gender"] = df["gender"].replace({"male": 1, "female":0})
df["children"] = df["children"].replace({"yes": 1, "no": 0})
df["affairs_binary"] = df["affairs"].apply(lambda x: 0 if x == 0 else 1)
print(df.head(500))

     affairs  gender   age  yearsmarried  children  religiousness  education  \
0          0       1  37.0        10.000         0              3         18   
1          0       0  27.0         4.000         0              4         14   
2          0       0  32.0        15.000         1              1         12   
3          0       1  57.0        15.000         1              5         18   
4          0       1  22.0         0.750         0              2         17   
..       ...     ...   ...           ...       ...            ...        ...   
495       12       1  47.0        15.000         1              4         17   
496       12       0  42.0        15.000         1              4         12   
497        7       1  27.0         4.000         0              3         14   
498        7       0  32.0         7.000         1              4         18   
499        1       1  32.0         0.417         1              3         12   

     occupation  rating  affairs_binary

## Descriptive analyisis

### Investigate what the ratio is between men cheating and woman cheating and ratio within class of not cheating

Use plotly library to display pie-charts

In [29]:
male_affairs = df[(df["gender"] == 1) & (df["affairs_binary"] == 1)].shape[0]
female_affairs = df[(df["gender"] == 0) & (df["affairs_binary"] == 1)].shape[0]
male_noaffairs = df[(df["gender"] == 1) & (df["affairs_binary"] == 0)].shape[0]
female_noaffairs = df[(df["gender"] == 0) & (df["affairs_binary"] == 0)].shape[0]

male_cheating_per = male_affairs / (male_affairs + male_noaffairs)
male_notcheating_per = male_noaffairs / (male_affairs + male_noaffairs)
female_cheating_per = female_affairs / (female_affairs + female_noaffairs)
female_notcheating_per = female_noaffairs / (female_affairs + female_noaffairs)

male_female_cheating_ratio = male_cheating_per/female_cheating_per
male_notcheating_cheating_ratio = male_notcheating_per/male_cheating_per
female_notcheating_cheating_ratio = female_notcheating_per/female_cheating_per

print("Ratio of men cheating and women cheating:", male_female_cheating_ratio)
print("Ratio of men not cheating:", male_notcheating_cheating_ratio)
print("Ratio of women not cheating:", female_notcheating_cheating_ratio)

labels = ['Ratio between men cheating and women cheating', 'Ratio between men not cheating and men cheating', 'Ratio between women not cheating and women cheating']
values = [male_female_cheating_ratio, male_notcheating_cheating_ratio, female_notcheating_cheating_ratio]
fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.show()


Ratio of men cheating and women cheating: 1.1931818181818181
Ratio of men not cheating: 2.666666666666667
Ratio of women not cheating: 3.3750000000000004


### Create a function that plots the relationship between the percentage cheating for women and men per category (and not cheating)

So for example:

plotPercentageCheating(age), takes as argument the age column.
It should plot for each category of age (<20, 20-24, ...) the  percentage of cheating male vs cheating female

Think of yourself how you want to visualize it such that it's easy for a reader to interpretate the results.

Note, make the visuals labels appearing, but it should be encoded back to the original values,

So education should be not 18 (someone who looks at this doesn't know what 18 is), it should be displayed master's degree.

In [30]:
df = pd.read_csv("Affairs.csv")
df["gender"] = df["gender"].replace({"male": 1, "female":0})
df["children"] = df["children"].replace({"yes": 1, "no": 0})
df["affairs_binary"] = df["affairs"].apply(lambda x: 0 if x == 0 else 1)
#mapping
age_mapping = {17.5: '<20', 22: '20-24', 27: '25-29', 32: '30-34', 37: '35-39', 42: '40-44', 47: '45-49', 52: '50-54', 57: '55+'}
yearsmarried_mapping = {0.125: '3 months or less', 0.417: '4-6 months', 0.75: '6 months-1 year', 1.5: '1-2 years', 4: '3-5 years', 7: '6-8 years', 10: '9-11 years', 15: '12+ years'}
children_mapping = {0: 0, 1: 1}
religiousness_mapping = {1: 'anti', 2: 'not at all', 3: 'slightly', 4: 'somewhat', 5: 'very'}
education_mapping = {9: 'grade school', 12: 'high school graduate', 14: 'some college', 16: 'college graduate', 17: 'some graduate work', 18: "master's degree", 20: 'Ph.D., M.D., or other advanced degree'}
occupation_mapping = {1: 'Unemployed and Unemployable', 2: 'Partly Unemployed and Partly Employed', 3: 'Unskilled Manuel Workers', 4: 'Semi-Skilled Workers', 5: 'Skilled and Technical Workers', 6: 'Managers abd supervisors', 7: 'Prestigious Professionals'}
rating_mapping = {1: 'very unhappy', 2: 'somewhat unhappy', 3: 'average', 4: 'happier than average', 5: 'very happy'}

df['age'] = df['age'].map(age_mapping)
df['yearsmarried'] = df['yearsmarried'].map(yearsmarried_mapping)
df['religiousness'] = df['religiousness'].map(religiousness_mapping)
df['education'] = df['education'].map(education_mapping)
df['occupation'] = df['occupation'].map(occupation_mapping)
df['rating'] = df['rating'].map(rating_mapping)
print(df)
print(df['age'].unique())

     Unnamed: 0  affairs  gender    age     yearsmarried  children  \
0             4        0       1  35-39       9-11 years         0   
1             5        0       0  25-29        3-5 years         0   
2            11        0       0  30-34        12+ years         1   
3            16        0       1    55+        12+ years         1   
4            23        0       1  20-24  6 months-1 year         0   
..          ...      ...     ...    ...              ...       ...   
596        1938        1       1  20-24        1-2 years         1   
597        1941        7       0  30-34       9-11 years         1   
598        1954        2       1  30-34       9-11 years         1   
599        1959        2       1  20-24        6-8 years         1   
600        9010        1       0  30-34        12+ years         1   

    religiousness             education  \
0        slightly       master's degree   
1        somewhat          some college   
2            anti  high school

In [32]:
mapping_dict = {
    'age': age_mapping,
    'yearsmarried': yearsmarried_mapping,
    'children': children_mapping,
    'religiousness': religiousness_mapping,
    'education': education_mapping,
    'occupation': occupation_mapping,
    'rating': rating_mapping
}

def plotPercentageCheating(columnname):
    categories = df[columnname].unique()

    data = {
        'Category': [],
        'Male Cheating': [],
        'Female Cheating': [],
        'Male Not Cheating': [],
        'Female Not Cheating': []
    }

    for category in categories:
        category_df = df[df[columnname] == category]

        total_males_females = category_df.shape[0]
        
        
        male_cheating_count = category_df[(category_df['gender'] == 1) & (category_df['affairs_binary'] == 1)].shape[0]
        female_cheating_count = category_df[(category_df['gender'] == 0) & (category_df['affairs_binary'] == 1)].shape[0]
        male_not_cheating_count = category_df[(category_df['gender'] == 1) & (category_df['affairs_binary'] == 0)].shape[0]
        female_not_cheating_count = category_df[(category_df['gender'] == 0) & (category_df['affairs_binary'] == 0)].shape[0]

        male_cheating_percentage = (male_cheating_count / total_males_females) * 100
        female_cheating_percentage = (female_cheating_count / total_males_females) * 100
        male_not_cheating_percentage = (male_not_cheating_count / total_males_females) * 100
        female_not_cheating_percentage = (female_not_cheating_count / total_males_females) * 100
        
        data['Category'].append(category)
        data['Male Cheating'].append(male_cheating_percentage)
        data['Female Cheating'].append(female_cheating_percentage)
        data['Male Not Cheating'].append(male_not_cheating_percentage)
        data['Female Not Cheating'].append(female_not_cheating_percentage)
        
    result_df = pd.DataFrame(data)
    
    fig = px.bar(result_df, x='Category', y=['Male Cheating', 'Female Cheating', 'Male Not Cheating', 'Female Not Cheating'],
            title='Percentage of Cheating and Not Cheating by Category',
            labels={'Category': columnname, 'value': 'Percentage'},
            barmode='group'
            )
    fig.update_xaxes(categoryorder='array', categoryarray=list(mapping_dict[columnname].values()))

    fig.show()      
pass

In [33]:
for column in ['age', 'yearsmarried', 'children', 'religiousness', 'education', 'occupation', 'rating']:
    plotPercentageCheating(column)

## Linear Regression

### Run a linear regression 
save the results of the coefficients (and intercept) of each feature in a dataframe named df_coefficients
test also for significance of each feature.

The df_coefficients should have two columns, the first one with the names, second with the values.

In [12]:
df = pd.read_csv("Affairs.csv")
df["gender"] = df["gender"].replace({"male": 1, "female":0})
df["children"] = df["children"].replace({"yes": 1, "no": 0})
df["affairs_binary"] = df["affairs"].apply(lambda x: 0 if x == 0 else 1)
X = df[["gender", "age", "yearsmarried", "children", "religiousness", "education", "occupation", "rating"]]
y = df['affairs_binary'] 

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

coefficients = model.params
p_values = model.pvalues

df_coefficients = pd.DataFrame({'Name': coefficients.index, 'Coefficient': coefficients.values})
print(df_coefficients)

            Name  Coefficient
0          const     0.736107
1         gender     0.045201
2            age    -0.007420
3   yearsmarried     0.015981
4       children     0.054487
5  religiousness    -0.053698
6      education     0.003078
7     occupation     0.005913
8         rating    -0.087455


# Coding Part 2

Previously it was a more guided coding part. Now comes the more open ended question and exploration.

We used affairs_binary only for part 1. Now we are focusing on the original data of affairs, where the value can also be greater than 1. 

Investigate what factors/drivers lead to a higher amount of affairs for men and women indivudally.

How you investigate and display the results is up to yourself.

In [24]:
df = pd.read_csv("Affairs.csv")
df["gender"] = df["gender"].replace({"male": 1, "female":0})
df["children"] = df["children"].replace({"yes": 1, "no": 0})

df_male = df[df['gender'] == 1]
df_female = df[df['gender'] == 0]

def regression(matrix): 
    X = matrix[["age", "yearsmarried", "children", "religiousness", "education", "occupation", "rating"]]
    y = matrix['affairs'] 
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    coefficients = model.params
    p_values = model.pvalues
    
    df_coefficients = pd.DataFrame({'Name': coefficients.index, 'Coefficient': coefficients.values, 'P-values': p_values})
    print(df_coefficients)
pass

for matrix in [df_male,df_female]:
    regression(matrix)
    

age_mapping = {17.5: '<20', 22: '20-24', 27: '25-29', 32: '30-34', 37: '35-39', 42: '40-44', 47: '45-49', 52: '50-54', 57: '55+'}
yearsmarried_mapping = {0.125: '3 months or less', 0.417: '4-6 months', 0.75: '6 months-1 year', 1.5: '1-2 years', 4: '3-5 years', 7: '6-8 years', 10: '9-11 years', 15: '12+ years'}
children_mapping = {0: 0, 1: 1}
religiousness_mapping = {1: 'anti', 2: 'not at all', 3: 'slightly', 4: 'somewhat', 5: 'very'}
education_mapping = {9: 'grade school', 12: 'high school graduate', 14: 'some college', 16: 'college graduate', 17: 'some graduate work', 18: "master's degree", 20: 'Ph.D., M.D., or other advanced degree'}
occupation_mapping = {1: 'Unemployed and Unemployable', 2: 'Partly Unemployed and Partly Employed', 3: 'Unskilled Manuel Workers', 4: 'Semi-Skilled Workers', 5: 'Skilled and Technical Workers', 6: 'Managers abd supervisors', 7: 'Prestigious Professionals'}
rating_mapping = {1: 'very unhappy', 2: 'somewhat unhappy', 3: 'average', 4: 'happier than average', 5: 'very happy'}

df['age'] = df['age'].map(age_mapping)
df['yearsmarried'] = df['yearsmarried'].map(yearsmarried_mapping)
df['religiousness'] = df['religiousness'].map(religiousness_mapping)
df['education'] = df['education'].map(education_mapping)
df['occupation'] = df['occupation'].map(occupation_mapping)
df['rating'] = df['rating'].map(rating_mapping)

mapping_dict = {
    'age': age_mapping,
    'yearsmarried': yearsmarried_mapping,
    'children': children_mapping,
    'religiousness': religiousness_mapping,
    'education': education_mapping,
    'occupation': occupation_mapping,
    'rating': rating_mapping
}

def plotPercentageCheating(columnname):
    categories = df[columnname].unique()

    data = {
        'Category': [],
        'Male Cheating': [],
        'Female Cheating': [],
    }

    for category in categories:
        category_df = df[df[columnname] == category]
        
        male_cheating_count = category_df[category_df['gender'] == 1]['affairs'].sum()
        female_cheating_count = category_df[category_df['gender'] == 0]['affairs'].sum()                
        data['Category'].append(category)
        data['Male Cheating'].append(male_cheating_count)
        data['Female Cheating'].append(female_cheating_count)
        
    result_df = pd.DataFrame(data)
    
    fig = px.bar(result_df, x='Category', y=['Male Cheating', 'Female Cheating'],
            title='Total numbers of cheating for male and female',
            labels={'Category': columnname, 'value': 'Total count'},
            barmode='group'
            )
    fig.update_xaxes(categoryorder='array', categoryarray=list(mapping_dict[columnname].values()))

    fig.show()      
pass

for column in ['age', 'yearsmarried', 'children', 'religiousness', 'education', 'occupation', 'rating']:
    plotPercentageCheating(column)

                        Name  Coefficient  P-values
const                  const     5.048775  0.003136
age                      age    -0.047072  0.107653
yearsmarried    yearsmarried     0.174989  0.001887
children            children    -0.125297  0.794419
religiousness  religiousness    -0.464242  0.003597
education          education     0.082409  0.381785
occupation        occupation    -0.044626  0.821659
rating                rating    -0.775013  0.000021
                        Name  Coefficient  P-values
const                  const     7.155770  0.000034
age                      age    -0.053597  0.145802
yearsmarried    yearsmarried     0.166750  0.009529
children            children    -0.144824  0.783394
religiousness  religiousness    -0.510756  0.001617
education          education    -0.106136  0.273079
occupation        occupation     0.144485  0.162197
rating                rating    -0.661380  0.000072



For male, the factors that led to higher amount of affairs are yearsmarried and education. As we can see from the linear regression output, an unit increase in yearsmarried (for example, moving from 3 months or less to 4-6 months) leads to a 0.17 time increase of having an affair on average. As for education, an unit increase in education (for example, moving from grade school to high school graduate) leads to a 0.08 time increase of having an affair on average. From the P-value we know that only yearsmarried is significant on a 5% significance level.

Similarly, the factors that led to higher amount of affairs for female are yearsmarried and occupation. From the linear regression output, an unit increase in yearsmarried leads to a 0.17 time increase of having an affair on average whereas an unit increase in occupation(for example, moving from Unemployed and Unemployable to Partly Unemployed and Partly Employed) leads to a 0.14 time increase of having an affair on average. Again, from P-value we know only yearsmarried is significant on a 5% significance level.